### 코드 전체 흐름
1. 문서의 내용을 읽는다
2. 문서를 쪼갠다.
- 토큰수 초과로 답변을 생성하지 못할 수 있고
- 문서가 길면(Input이 길면) 답변 생성이 오래걸림
3. 임베딩 -> 벡터 DB에 저장
4. 질문이 있을 때, 벡터 DB에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [1]:
%pip install --upgrade --quiet  docx2txt langchain-community
%pip install -qU langchain-text-splitters
%pip install -U langchain langchainhub --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,    # 문서를 쪼갤 때 하나의 chunk 가 가지는 토큰 수
    chunk_overlap=200,  # 텍스트를 분할할 때, 각 chunk(조각) 사이에 중복되는 부분의 토큰 수
)

loader = Docx2txtLoader('tax.docx')
document = loader.load()                                           #단순히 문서를 읽을 때
document_list = loader.load_and_split(text_splitter=text_splitter) # 문서를 쪼갤 때

In [3]:
print('쪼개기 전 문서 : ',len(document))
print('쪼갠 문서 : ',len(document_list))

쪼개기 전 문서 :  1
쪼갠 문서 :  220


In [4]:
from dotenv import load_dotenv                # OpenAI API Key 사용을 위해 환경변수 등록
from langchain_openai import OpenAIEmbeddings # OpenAI 의 Embedding을 사용

load_dotenv() # 환경변수 불러오기

embeddings = OpenAIEmbeddings(model='text-embedding-3-large') # 기본모델은 002인 예전모델이다. 신규 모델을 사용하기 위해 추가

In [5]:
%pip install -qU langchain-pinecone pinecone-notebooks

Note: you may need to restart the kernel to use updated packages.


In [9]:
%pip uninstall -y pinecone-notebooks

Found existing installation: pinecone-notebooks 0.1.1
Uninstalling pinecone-notebooks-0.1.1:
  Successfully uninstalled pinecone-notebooks-0.1.1
Note: you may need to restart the kernel to use updated packages.


In [11]:
import os

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = 'tax-index'
pinecone_api_key = os.environ.get('PINECONE_API_KEY')

pc = Pinecone(api_key=pinecone_api_key)
database = PineconeVectorStore.from_documents(document_list, embeddings, index_name=index_name)

In [12]:
query = '연봉 3000만원인 직장인의 소득세는 얼마인가요?'
retrieved_docs = database.similarity_search(query, 1); # similarity_search() 함수는 유사도 검색을 위한 함수이다.

In [13]:
retrieved_docs

[Document(id='bd05bdfb-e1da-4430-ab56-5a1256cfb2e7', metadata={'source': 'tax.docx'}, page_content='나. 그 밖의 배당소득에 대해서는 100분의 14\n\n3. 원천징수대상 사업소득에 대해서는 100분의 3. 다만, 외국인 직업운동가가 한국표준산업분류에 따른 스포츠 클럽 운영업 중 프로스포츠구단과의 계약(계약기간이 3년 이하인 경우로 한정한다)에 따라 용역을 제공하고 받는 소득에 대해서는 100분의 20으로 한다.\n\n4. 근로소득에 대해서는 기본세율. 다만, 일용근로자의 근로소득에 대해서는 100분의 6으로 한다.\n\n5. 공적연금소득에 대해서는 기본세율\n\n5의2.제20조의3제1항제2호나목 및 다목에 따른 연금계좌 납입액이나 운용실적에 따라 증가된 금액을 연금수령한 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 각 목의 요건을 동시에 충족하는 때에는 낮은 세율을 적용한다.\n\n가. 연금소득자의 나이에 따른 다음의 세율\n\n\n\n나. 삭제<2014. 12. 23.>\n\n다. 사망할 때까지 연금수령하는 대통령령으로 정하는 종신계약에 따라 받는 연금소득에 대해서는 100분의 4\n\n5의3. 제20조의3제1항제2호가목에 따라 퇴직소득을 연금수령하는 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 연금 실제 수령연차 및 연금외수령 원천징수세율의 구체적인 내용은 대통령령으로 정한다.\n\n가. 연금 실제 수령연차가 10년 이하인 경우: 연금외수령 원천징수세율의 100분의 70\n\n나. 연금 실제 수령연차가 10년을 초과하는 경우: 연금외수령 원천징수세율의 100분의 60\n\n6. 기타소득에 대해서는 다음에 규정하는 세율. 다만, 제8호를 적용받는 경우는 제외한다.\n\n가. 제14조제3항제8호라목 및 마목에 해당하는 소득금액이 3억원을 초과하는 경우 그 초과하는 분에 대해서는 100분의 30\n\n나. 제21조제1항제18호 및 제21호에 따른

In [14]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [15]:
# 기존에는 질문만 날렸지만 이제는 질문+문서 를 질의한다. 즉, 프롬프트 작성
prompt = f"""[Identity]
- 당신은 현존하는 세계 최고의 한국 소득세 전문가 입니다
- [Context]를 참고해서 사용자의 질문에 답변해주세요

[Context]
{retrieved_docs}

Question: {query}
"""


In [16]:
ai_message = llm.invoke(prompt)

In [17]:
ai_message.content

'연봉 3000만원인 직장인의 소득세를 계산하기 위해서는 한국의 소득세 기본세율을 참고해야 합니다. 한국의 소득세는 누진세율 구조를 가지고 있으며, 소득 구간에 따라 다른 세율이 적용됩니다. 2023년 기준으로 한국의 소득세 기본세율은 다음과 같습니다:\n\n- 1,200만원 이하: 6%\n- 1,200만원 초과 ~ 4,600만원 이하: 15%\n- 4,600만원 초과 ~ 8,800만원 이하: 24%\n- 8,800만원 초과 ~ 1억5천만원 이하: 35%\n- 1억5천만원 초과 ~ 3억원 이하: 38%\n- 3억원 초과 ~ 5억원 이하: 40%\n- 5억원 초과: 42%\n\n연봉 3000만원의 경우, 1,200만원까지는 6%의 세율이 적용되고, 나머지 1,800만원에 대해서는 15%의 세율이 적용됩니다.\n\n계산:\n1. 1,200만원 × 6% = 72만원\n2. (3,000만원 - 1,200만원) × 15% = 1,800만원 × 15% = 270만원\n\n따라서, 총 소득세는 72만원 + 270만원 = 342만원입니다.\n\n이 계산은 기본적인 소득세만을 고려한 것이며, 실제로는 각종 공제 및 세액공제를 적용하여 최종적으로 납부해야 할 세금이 달라질 수 있습니다. 공제를 적용받을 수 있는 항목에 따라 실질적인 납부 세액은 더 낮아질 수 있습니다.'

In [18]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

C:\Users\DataLogics\miniconda3\envs\llm_lab\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [19]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [20]:
# QA Chain 만들기
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt":prompt}
)

In [21]:
ai_message = qa_chain({"query" : query})

C:\Users\DataLogics\AppData\Local\Temp\ipykernel_49136\914447175.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ai_message = qa_chain({"query" : query})


In [22]:
ai_message

{'query': '연봉 3000만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉 3000만원인 직장인의 소득세는 기본세율에 따라 계산됩니다. 총급여액이 3300만원 이하인 경우, 소득세액에서 74만원을 공제받을 수 있습니다. 정확한 소득세율과 공제 금액은 개인의 상황에 따라 달라질 수 있습니다.'}

In [23]:
%conda list pydantic

# packages in environment at C:\Users\DataLogics\miniconda3\envs\llm_lab:
#
# Name                    Version                   Build  Channel
pydantic                  2.10.3                   pypi_0    pypi
pydantic-core             2.27.1                   pypi_0    pypi
pydantic-settings         2.6.1                    pypi_0    pypi

Note: you may need to restart the kernel to use updated packages.


In [24]:
%conda list langchain

# packages in environment at C:\Users\DataLogics\miniconda3\envs\llm_lab:
#
# Name                    Version                   Build  Channel
langchain                 0.3.11                   pypi_0    pypi
langchain-chroma          0.1.4                    pypi_0    pypi
langchain-community       0.3.11                   pypi_0    pypi
langchain-core            0.3.24                   pypi_0    pypi
langchain-openai          0.2.12                   pypi_0    pypi
langchain-pinecone        0.2.0                    pypi_0    pypi
langchain-text-splitters  0.3.2                    pypi_0    pypi
langchain-upstage         0.4.0                    pypi_0    pypi
langchainhub              0.1.21                   pypi_0    pypi

Note: you may need to restart the kernel to use updated packages.
